In [85]:
from ortools.linear_solver import pywraplp
import pandas as pd
from IPython.display import display
"""NUI Galway CT5141 Assignment 1 : Unit Commitment

Student name(s): Galvin Venugopal, Atharva Kulkarni
Student ID(s):     (20235245)        (20231773)

We have a country with 10 generators of 4 types: Hydroelectric, Solid fuel, Gas, Solar
* Minimize cost of electricity production per hour
* How much electricity does each generator produce per hour

Decision variables

* A_1 to A_24 is the quantity of electricity produced by generator A during each hour of the day
* B_1 to B_24 is the quantity of electricity produced by generator B during each hour of the day
* C_1 to C_24 is the quantity of electricity produced by generator C during each hour of the day
* D_1 to D_24 is the quantity of electricity produced by generator D during each hour of the day
* E is the quantity of electricity produced by generator E during each hour of the day
* F is the quantity of electricity produced by generator F during each hour of the day
* G is the quantity of electricity produced by generator G during each hour of the day
* H_1 to H_24 is the quantity of electricity produced by generator H during each hour of the day
* I_1 to I_24 is the quantity of electricity produced by generator I during each hour of the day
* J_1 to J_24 is the quantity of electricity produced by generator J during each hour of the day

Objective Function

To minimize the cost of the electricity generated
(1.4*(A_1+...+A_24))+(1.4*(B_1+...+B_24))+(1.4*(C_1+...+C_24))+(1.4*(D_1+...+D_24))+(4.4*(E))+
(4.4*(F))+(4.4*(G))+(9.1*(H_1+...+H_24))+(6.6*(I_1+...+I_24))+(6.6*(J_1+...+J_24))

Constraints

10  <= A_1..A_24 <= 100
10  <= B_1..B_24 <= 80
10  <= C_1..C_24 <= 60
1   <= D_1..D_24 <= 10
100 <= E <= 900
100 <= F <= 600
10  <= G <= 100
100 <= H_1..H_24 <= 400
0   <= I_1..I_24 <= 70
0   <= J_1..J_24 <= 20

All Solar Generators (I,J) produce different levels based on the amount of sunlight available.
Total electricity generated in an hour by all the generators should be equal to the demand

"""
def get_problem_details():

    energy_generators = pd.read_csv("generator_info.csv")
    energy_generators.columns = ["Name","Type","Lower Bound","Upper Bound","Cost/MW","CO2/MW"]
    # display(energy_generators.head(100))

    names = energy_generators["Name"].values.tolist()
    lower_bounds = energy_generators["Lower Bound"].values.tolist()
    upper_bounds = energy_generators["Upper Bound"].values.tolist()
    costs_per_mw = energy_generators["Cost/MW"].values.tolist()

    demand = pd.read_csv("demand.csv",header=None)
    demand.columns = ["Hourly Demand"]
    display(demand.head(100))
    hourly_demand = demand["Hourly Demand"].values.tolist()

    solar_capacity = pd.read_csv("solar_curve.csv",header=None)
    solar_capacity.columns = ["Hourly Solar Capacity"]
    hourly_solar_capacity = solar_capacity["Hourly Solar Capacity"].values.tolist()
    # display(solar_capacity.head(100))

    return energy_generators, hourly_demand, hourly_solar_capacity, names, lower_bounds, upper_bounds, costs_per_mw
    
def minimize_cost():

    energy_generators, hourly_demand, hourly_solar_capacity, names, lower_bounds, upper_bounds, costs_per_mw = get_problem_details()
    solver_variables = []

    # Create an LP Solver object
    solver = pywraplp.Solver('Generator', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # Create All Decision Variables
    for name,lb,ub in zip(names, lower_bounds, upper_bounds):
        if(name == "E" or name == "F" or name == "G"):
            solver_variables.append(solver.NumVar(lb, ub, f"{name}"))
        else:
            solver_variables.append([solver.NumVar(lb, ub, f"{name}{i}") for i in range(24)])

    print(solver_variables)
    for solver_var in solver_variables:
        for i, (solar_coefficient, demand) in enumerate(zip(hourly_solar_capacity, hourly_demand)):
            solver.Add(
            solver_variables[0][i] + 
            solver_variables[1][i] + 
            solver_variables[2][i] +
            solver_variables[3][i] +
            solver_variables[4] +
            solver_variables[5] +
            solver_variables[6] +
            solver_variables[7][i] +
            solar_coefficient * solver_variables[8][i] +
            solar_coefficient * solver_variables[9][i] == demand)
        
    #Constraints
    #for j in range(num_hours):
    # for ai, bi, ci, di, hi, ii, ji, sol_i, dem_i in zip(A,B,C,D,H,I,J,Solarcurve, Demand):
    #     #solver.Add(A[j] + B[j] + C[j] + D[j] + E + F + G + H[j] + Solarcurve[j]*I[j] + Solarcurve[j]*H[j] == Demand[j])
    #     solver.Add(ai + bi + ci + di + E + F + G + hi + sol_i*ii + sol_i*ji == dem_i)
    print("Num of Constraints: ", solver.NumConstraints())
    
    objective = solver.Objective()

    for j in range(24):
        objective.SetCoefficient(solver_variables[0][j], costs_per_mw[0])
        objective.SetCoefficient(solver_variables[1][j], costs_per_mw[1])
        objective.SetCoefficient(solver_variables[2][j], costs_per_mw[2])
        objective.SetCoefficient(solver_variables[3][j], costs_per_mw[3])
        objective.SetCoefficient(solver_variables[7][j], costs_per_mw[7])
        objective.SetCoefficient(solver_variables[8][j], costs_per_mw[8])
        objective.SetCoefficient(solver_variables[9][j], costs_per_mw[9])

    objective.SetCoefficient(solver_variables[4],costs_per_mw[4])
    objective.SetCoefficient(solver_variables[5],costs_per_mw[5])
    objective.SetCoefficient(solver_variables[6],costs_per_mw[6])

    objective.SetMinimization()
    # for solver_var, cost in zip(solver_variables, cost_per_mw):

    # objective.SetCoefficient(A0, )
    # #Objective
    # objective = solver.Objective()
    
    
    # objective.SetCoefficient(E, Coeff[4])
    # objective.SetCoefficient(F, Coeff[5])
    # objective.SetCoefficient(G, Coeff[6])
    # objective.SetMinimization()
    
    result = solver.Solve()
    if result == solver.OPTIMAL:
        for v in solver.variables():
            print(f"{v.name()} = {v.solution_value():.5}; coefficient {objective.GetCoefficient(v):.5}")
            
        print(f"Optimal objective value = {solver.Objective().Value()}")
        
        # print_solution_and_sensitivity(solver,objective)

    else:
        print("Problem is not feasible")




minimize_cost()

,Hourly Demand
0,1461
1,1446
2,1446
3,1438
4,1425
5,1420
6,1467
7,1503
8,1507
9,1504


[[A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22, A23], [B0, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, B12, B13, B14, B15, B16, B17, B18, B19, B20, B21, B22, B23], [C0, C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12, C13, C14, C15, C16, C17, C18, C19, C20, C21, C22, C23], [D0, D1, D2, D3, D4, D5, D6, D7, D8, D9, D10, D11, D12, D13, D14, D15, D16, D17, D18, D19, D20, D21, D22, D23], E, F, G, [H0, H1, H2, H3, H4, H5, H6, H7, H8, H9, H10, H11, H12, H13, H14, H15, H16, H17, H18, H19, H20, H21, H22, H23], [I0, I1, I2, I3, I4, I5, I6, I7, I8, I9, I10, I11, I12, I13, I14, I15, I16, I17, I18, I19, I20, I21, I22, I23], [J0, J1, J2, J3, J4, J5, J6, J7, J8, J9, J10, J11, J12, J13, J14, J15, J16, J17, J18, J19, J20, J21, J22, J23]]
Num of Constraints:  240
A0 = 51.0; coefficient 1.4
A1 = 36.0; coefficient 1.4
A2 = 36.0; coefficient 1.4
A3 = 28.0; coefficient 1.4
A4 = 15.0; coefficient 1.4
A5 = 10.0; coefficient 1.4
A6 = 57.0; coeffici

In [86]:
print(51 + 10 + 10 + 1 + 589 + 600 + 100 + 100 + 0 + 0)

1461


In [ ]:
def print_solution_and_sensitivity(solver, objective):
    """This function prints the solution and extracts all the sensitivity information
    that is easily available in OR-Tools. If the problem is IP, no
    sensitivity information is available, so it just prints what it can."""

    # is this a continuous problem? we can do more sensitivity
    # analysis if so.
    continuous_problem = all(v.Integer() == False for v in solver.variables())

    # the *reduced cost* for a variable is the change in objective
    # coefficient for the variable which would be required to move the
    # location of the optimum
    
        
    for c, a in zip(solver.constraints(), solver.ComputeConstraintActivities()):
        eps = 0.0000001
        # a constraint is *binding* if it is actually preventing the
        # optimum from improving -- the constraint line goes through
        # the optimum. another way to see it: the activity equals the
        # bound. another way: the dual is zero (the dual exists only
        # for continuous problems). we print a "*" for binding
        # constraints.
        if continuous_problem:
            binding = "* " if abs(c.DualValue()) > eps else "  "
        else:
            binding = "* " if (abs(c.ub() - a) < eps or abs(a - c.lb()) < eps) else "  "

        ctxt = " + ".join(f"{c.GetCoefficient(v):.5}*{v.name()}"
                          for v in solver.variables())
        
        # the *dual value* aka *shadow price* of a constraint is the
        # amount our profit could improve if the RHS of the constraint
        # would improve by 1 unit. for non-binding constraints, the
        # dual is 0. 

        if continuous_problem:

            if c.lb() == float("-inf"):
                # it is a <= constraint
                print(f"{binding} {c.name()}: {ctxt} = {a:.5} <= {c.ub():.5}; dual {c.DualValue():.5}; slack {c.ub() - a:.5}")
            elif c.ub() == float("inf"):
                # it is a >= constraint
                print(f"{binding} {c.name()}: {ctxt} = {a:.5} >= {c.lb():.5}; dual {c.DualValue():.5}; slack {a - c.lb():.5}")
            else:
                raise ValueError
                
        else:
            if c.lb() == float("-inf"):
                # it is a <= constraint
                print(f"{binding} {c.name()}: {ctxt} = {a:.5} <= {c.ub():.5}; slack {c.ub() - a:.5}")
            elif c.ub() == float("inf"):
                # it is a >= constraint
                print(f"{binding} {c.name()}: {ctxt} = {a:.5} >= {c.lb():.5}; slack {a - c.lb():.5}")
            else:
                raise ValueError